In [24]:
from frame_classification_process import frame_process
from keras.utils import to_categorical
import numpy as np
import os

curr_path = os.getcwd()
x_train_ind = np.load(curr_path+'/datasets/x_train_ind_above400.npy')
y_train = np.load(curr_path+'/datasets/y_train_mapped_above400.npy')
num_videos = len(y_train)
num_classes = len(np.unique(y_train))

 
num_videos = 1000 # parameter setting
if num_videos > len(y_train) or None: 
    num_videos = len(y_train)

    
x_train_ind = x_train_ind[:num_videos]
y_train = to_categorical(y_train[:num_videos], num_classes=num_classes)

fp = frame_process(num_videos, size = (80,80,3))
X = fp.process_updates_frameSeq_stacked(x_train_ind, y_train, 10, "c3d_X_train")

100%|██████████| 1000/1000 [01:51<00:00,  5.26it/s]


In [25]:
from models import models


model = models(num_classes, "c3d", num_frames = 10, reg = 0)
history = model.model.fit(X, y_train, epochs=30, batch_size=32, validation_split = 0.2, verbose = 1)


(10, 80, 80, 3)
(None, 4, 38, 38, 32)
(None, 2, 17, 17, 64)
(None, 1, 7, 7, 128)
(None, 256)
(None, 256)
(None, 10)
Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 3s - loss: 6.2494 - acc: 0.0975 - val_loss: 11.4986 - val_acc: 0.1000
Epoch 2/30
800/800 [==============================] - 2s - loss: 2.8962 - acc: 0.1113 - val_loss: 12.3264 - val_acc: 0.1400
Epoch 3/30
800/800 [==============================] - 2s - loss: 2.6805 - acc: 0.0938 - val_loss: 9.8190 - val_acc: 0.1000
Epoch 4/30
800/800 [==============================] - 3s - loss: 2.6338 - acc: 0.1175 - val_loss: 7.9450 - val_acc: 0.1000
Epoch 5/30
800/800 [==============================] - 3s - loss: 2.5336 - acc: 0.1050 - val_loss: 4.8469 - val_acc: 0.1000
Epoch 6/30
800/800 [==============================] - 2s - loss: 2.5412 - acc: 0.0925 - val_loss: 3.2546 - val_acc: 0.1000
Epoch 7/30
800/800 [==============================] - 2s - loss: 2.4993 - acc: 0.1162 - val_loss: 

In [19]:
print (X.shape)
y_train = np.load(curr_path+'/datasets/y_train_mapped_above400.npy')
y_train = y_train[:num_videos]
unique, counts = np.unique(y_train, return_counts=True)
y_count = dict(zip(unique, counts))
print (y_count)

(5, 10, 80, 80, 3)
{8: 1, 1: 1, 2: 1, 9: 2}


In [ ]:
# plot training history
# print(history.history.keys())
# plt.subplots(121)
plt.plot(history.history['acc'])
# # plt.plot(history.history['val_acc'])